La première partie du projet sera liée à l'étude des data


In [19]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
from scipy.signal import savgol_filter
from scipy.stats import zscore


file_path = "/Users/alicepriolet/Desktop/ML/epfl-bio-322-2024/train.csv"
data = pd.read_csv(file_path)

isNan = True # permet de choisir si la methode des Nan doit etre utilisee 
nb_max_Nan = 10
      

    
def process_Nan(data):
    # Compter le nombre total de NaN dans le fichier
    nombre_nan = data.isna().sum().sum()  
    print(f"Nombre total de NaN : {nombre_nan}")
    
    if nombre_nan >= nb_max_Nan:
        # Remplir les NaN avec la moyenne des colonnes si nombre_nan > nb_max_Nan 
        data.fillna(data.mean(), inplace=True)
    else:
        # Supprimer les lignes contenant des NaN si nombre_nan < nb_max_Nan
        data = data.dropna()  # Supprime les lignes avec au moins un NaN
        
        
 
def preprocess_categorical(data):
    """
    Préprocesser les colonnes catégoriques d'un DataFrame en utilisant pd.get_dummies.
    
    Args:
    - data (pd.DataFrame): Tableau de données à prétraiter.

    Returns:
    - pd.DataFrame: DataFrame avec colonnes catégoriques transformées en dummies.
    """
    # Identifier les colonnes de type 'object' (catégoriques)
    string_columns = data.select_dtypes(include=['object']).columns
    
    if not string_columns.empty:
        print(f"Colonnes catégoriques trouvées : {list(string_columns)}")
        # Appliquer pd.get_dummies pour transformer les colonnes catégoriques
        data = pd.get_dummies(data, columns=string_columns, drop_first=True)
        print("Les colonnes catégoriques ont été transformées en dummies.")
    else:
        print("Aucune colonne catégorique trouvée.")
    
    return data
 
 
if isNan == True: 
    process_Nan(data)
else :
    print('No Nan processing has been performed')
    
preprocess_categorical(data)       
print(data)
spectrum = data.iloc[:, 6:] # permet de standardiser les valeurs d infrarouge 
spectrum_filtered = pd.DataFrame(savgol_filter(spectrum, 7, 3, deriv = 2, axis = 0))
spectrum_filtered_standardized = zscore(spectrum_filtered, axis = 1)

#plt.plot(spectrum_filtered[0],spectrum_filtered[6])
#plt.show()
#print(spectrum_filtered_standardized)  # Statistiques descriptives

"""Preprocessing functions."""

import numpy as np



def variance_filter(data, threshold):
    """
    Applique un filtre de variance sur un fichier CSV en supprimant les colonnes avec faible variance.

    Args:
        file_path (str): Chemin du fichier CSV.
        threshold (float): Seuil de variance en dessous duquel les colonnes seront supprimées.
        output_file (str): Nom du fichier CSV où sauvegarder les données filtrées (par défaut : "filtered_data.csv").

    Returns:
        None
    """
    
    # Calculer la variance pour chaque colonne numérique
    variances = data.std(axis=0)  # Écart type (équivalent à la racine carrée de la variance)
    
    # Identifier les colonnes à supprimer
    del_cols = variances[variances <= threshold].index  # Noms des colonnes avec faible variance
    
    # Supprimer les colonnes identifiées
    filtered_data = data.drop(columns=del_cols)
    
    # Sauvegarder les données filtrées dans un nouveau fichier CSV
    #filtered_data.to_csv(filtered_data.csv, index=False)
    
    # Afficher les résultats
    print(f"Colonnes supprimées (variance <= {threshold}): {list(del_cols)}")
    print(f"Les données filtrées ont été sauvegardées dans '{output_file}'.")


def standardization(
    data: np.array, mean: np.array = None, std: np.array = None
) -> tuple[np.array, np.array, np.array]:
    """
    Standardize the data. If mean or std is None, we compute them.

    Args:
        data (np.array): data matrix (N,D)
        mean (np.array, optional): means for standardization. Defaults to None.
        std (np.array, optional): standard deviations for standardization. Defaults to None.

    Returns:
        tuple[np.array, float, float]: standardized data matrix, columns means and standard deviations
    """
    if (mean is None) or (std is None):
        mean = np.mean(data, axis=0)
        std = np.std(data, axis=0)

    return (data - mean) / std, mean, std


def add_bias_term(x: np.array) -> np.array:
    """
    Add a column of ones on the left of the data matrix

    Args:
        x (np.array): data matrix (N,D)

    Returns:
        np.array: data matrix with bias terms
    """
    num_samples = x.shape[0]
    tx = np.c_[np.ones(num_samples), x]

    return tx


def preprocess_data(
    x_tr: np.array, x_te: np.array, nan_threshold: float = 0, var_threshold: float = 0
) -> tuple[np.array, np.array]:
    """
    Call preprocessing functions

        x_tr (np.array): raw x_train (N,)
        x_te (np.array): row x_test (N_test,)
        nan_threshold (float, optional): threshold for Nan values. Defaults to 0.
        var_threshold (float, optional): threshold for variance. Defaults to 0.

    Returns:
        tuple[np.array, np.array]: x_train and x_test after preprocessing
    """
    # Remove non relevant features :
    x_tr, del_features = nan_filter(x_tr, threshold=nan_threshold)
    x_te = np.delete(x_te, del_features, axis=1)

    # Replace NaN values and remove zero variance features in training set :
    x_tr = fill_nan_mean(x_tr)
    x_tr, del_features = variance_filter(x_tr, threshold=var_threshold)

    # Remove same features and replace NaN values in testing set :
    x_te = np.delete(x_te, del_features, axis=1)
    x_te = fill_nan_mean(x_te)

    # Standardize data :
    x_tr, mean_tr, std_tr = standardization(x_tr)
    x_te, _, _ = standardization(x_te, mean_tr, std_tr)

    # Add Bias term :
    tx_tr = add_bias_term(x_tr)
    tx_te = add_bias_term(x_te)

    return tx_tr, tx_te


Nombre total de NaN : 0
Colonnes catégoriques trouvées : ['sample_name', 'device_serial', 'substance_form_display', 'measure_type_display', 'prod_substance']
Les colonnes catégoriques ont été transformées en dummies.
          sample_name device_serial  substance_form_display  \
0            11140009    M1-1000112      Homogenized Powder   
1     22.0401-P002.02    M1-1000109  Non homogenized powder   
2     22.0117-P001.02    M1-1000100  Non homogenized powder   
3        20.0163-P009      N1-00196             Unspecified   
4        19.0286-P005      N1-00196             Unspecified   
...               ...           ...                     ...   
1295  22.0267-P001.01    M1-1000100      Homogenized Powder   
1296         11060000    M1-1000144      Homogenized Powder   
1297     22.0368-P002    M1-1000130      Homogenized Powder   
1298  22.0342-P002.04    M1-1000109      Homogenized Powder   
1299  22.0401-P001.04    M1-1000109      Homogenized Powder   

     measure_type_display 